### 导入库

In [1]:
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import math
from scipy import stats
from sklearn.utils.multiclass import type_of_target

## 1. 对数据集自动清理

预定义函数

In [2]:
# 删除重复列
def drop_dup_cols(input_dataframe):
    input_dataframe.T.drop_duplicates().T
    return input_dataframe

# 删除含有空值的行
def drop_nans(input_dataframe):
    input_dataframe.dropna(inplace=True)
    return input_dataframe

# 删除指定列
def drop_col(input_dataframe, column):
    input_dataframe = input_dataframe.drop([column],axis=1)
    print("成功删除特征：",column)
    return input_dataframe

# 删除异常值
def drop_outfiers(input_dataframe, column):
    drop_indices = []
    percent_25 = np.percentile(input_dataframe[column],25)
    percent_75 = np.percentile(input_dataframe[column],75)
    normal_range = (percent_75 - percent_25) * 3
    outlier_indexs = input_dataframe[(input_dataframe[column] < percent_25 - normal_range) | (input_dataframe[column] > percent_75 + normal_range)].index
    drop_indices.extend(list(outlier_indexs ))
    input_dataframe = input_dataframe.drop(drop_indices)
    return input_dataframe, percent_25, percent_75

# 查找指定比例的值作为异常值，用极大极小值代替
def replace_outfiers_by_percent(input_dataframe, column, percent):
    drop_indices = []
    percent_min = np.percentile(input_dataframe[column],100-percent)
    percent_max = np.percentile(input_dataframe[column],percent)
    input_dataframe[column] = input_dataframe[column].map(lambda x: percent_max if x > percent_max else x)
    input_dataframe[column] = input_dataframe[column].map(lambda x: percent_min if x < percent_min else x)
    return input_dataframe,percent_min,percent_max

# 对指定特征使用one hot 编码
def one_hot_encoder_feature(input_dataframe, column):
    one_hot_column = pd.get_dummies(input_dataframe[column])
    input_dataframe = pd.concat([one_hot_column,input_dataframe],axis=1)
    input_dataframe = input_dataframe.drop([column],axis=1)
    return input_dataframe

# 对指定特征使用label encoder 编码
def label_encoder(input_dataframe, column):
    column_encoder = LabelEncoder().fit(input_dataframe[column].values)
    input_dataframe[column] = column_encoder.transform(input_dataframe[column].values)
    return input_dataframe

# 对特征归一化处理
def normalize(input_dataframe, column, norm_max, norm_min):
    '''
    norm_min: int
        归一化后范围的下界 （默认：0）
    norm_max: int
        归一化后范围的上界 （默认：1）
    '''
    min_x = min(input_dataframe[column])
    max_x = max(input_dataframe[column])
    range_x = max_x - min_x
    range_norm = norm_max - norm_min
    input_dataframe[column] = input_dataframe[column].map(lambda x: x/range_x * range_norm)
    return input_dataframe, min_x, max_x

# 对空值进行处理
def deal_non(input_dataframe, column, null_method, similarity_features, remain_features):
    if null_method:
        if null_method not in ['mean','mode','median','similarity']:
            print("请输入正确的空缺值处理方法")
            return

    # 若指定以平均数替代
    if null_method == 'mean':
        replace_value = input_dataframe[column].mean()
        input_dataframe[column].fillna(replace_value, inplace=True)

    # 若指定以中位数替代
    elif null_method == 'median':
        replace_value = input_dataframe[column].median()
        input_dataframe[column].fillna(replace_value, inplace=True)

    # 若指定以众数替代
    elif null_method == 'mode':
        replace_value = input_dataframe[column].mode()
        input_dataframe[column].fillna(input_dataframe[column].mode(), inplace=True)

    # 若指定以相似度替代
    elif null_method == 'similarity':
        if not similarity_features:
            print("请输入相似度参考特征")
            return

        # 输入的特征不属于原特征
        elif set(remain_features) > set(similarity_features) == False:
            print("请输入正确的特征名")

        # 根据提供特征的相似度替换空缺值
        else:
            if column not in similarity_features and input_dataframe[column].dtype == 'int64' or input_dataframe[column].dtype == 'float64':
                input_dataframe[column].fillna(data.groupby(similarity_features)[column].transform('mean'),inplace=True)
            else:
                input_dataframe[column].fillna(input_dataframe[column].mode(), inplace=True)
    return input_dataframe, replace_value

# 识别年份，特殊处理，转换为object类型，之后转换为枚举类
def detect_year(input_dataframe, column):
    tmp_year = column.lower()
    pattern = r"year"
    if re.search(pattern,tmp_year):
        input_dataframe[column] = input_dataframe[column].astype('object')
    return input_dataframe

# 识别编号，特殊处理
def detect_number(input_dataframe, column):
    tmp_no = column.lower()
    pattern = r"num|no|number"
    if re.search(pattern,tmp_no):
        input_dataframe[column] = input_dataframe[column].astype('str')
    return input_dataframe

In [3]:
def autoclean(input_dataframe, normalization = False, drop_nans=False,
              nans_convert_binary = False, null_method = 'mode', 
              similarity_features = None, enum_continuous = False, 
              one_hot_encoder = False, no_one_hot_features = None, 
              norm_max = 1, norm_min = 0, 
              replace_outfiers_by_percent = False, percent = 95):
    """对数据集进行标准化数据预处理
    
    ----------
    input_dataframe: pandas.DataFrame
        输入的数据集
    normalization: bool
        对数据归一化 （默认: False）
    drop_nans: bool
        删除所有包含NaN的行 (默认: False)
    nans_convert_binary: bool
        把存在的值替换为1，不存在替换为0
    null_method: string
        空缺值处理方法 （默认: mode）
        可选 mode, average, median, similarity
    similarity_features: list
        以list中的特征作为相似度处理空缺值
    enum_continuous: bool
        把连续型变量转换为枚举类 （默认: True）
    one_hot_encoder: bool
        使用OneHotEncoder编码方式. (默认: False)
    no_one_hot_features: list
        指定不需要使用OneHotEncoder编码的特征 (默认：None)
    norm_min: int
        归一化后范围的下界 （默认：0）
    norm_max: int
        归一化后范围的上界 （默认：1）
    replace_outfiers_by_percent: bool
        用极大极小值代替指定头尾百分比的值 （默认：False）
    percent: float
        仅当replace_outfiers_by_percent为True时有效，前percent百分比
        和后（100-percent）百分比的值被认为异常值
    Returns
    ----------
    input_dataframe: pandas.DataFrame
        清理完成后的数据集
    """
    # 剩下的特征名
    remain_features = input_dataframe.columns.tolist()
        
    # 删除重复的特征
    input_dataframe = drop_dup_cols(input_dataframe)
    
    # 删除包含NaN的行
    if drop_nans:
        input_dataframe = drop_nans(input_dataframe)
        remain_features.remove(column)
    
    # 对于每一个特征
    for column in input_dataframe.columns.values:
        print("当前特征", column)
        
        # 分别计算记录总条数，种类数，空值数，0的数量，唯一值比例，空值比例，值为0的比例
        count = input_dataframe[column].count()
        unique_sum = len(input_dataframe[column].unique())
        empty = input_dataframe[column].isnull().sum()
        zeros = (input_dataframe[column] == 0).sum()
        unique_ratio = unique_sum / count
        empty_ratio = empty / count
        zeros_ratio = zeros/count
        
        # 判断是否为唯一标示特征，即当特征的每一列都不同，特征无效
        if unique_ratio > 0.7:
            input_dataframe = drop_col(input_dataframe, column)
            remain_features.remove(column)
            continue;
            
        # 当特征每一列都相同，特征无效
        if unique_sum  == 1:
            input_dataframe = drop_col(input_dataframe, column)
            remain_features.remove(column)
            continue;
        
        # 对于空值比例较高的特征（> 60%）
        if empty_ratio > 0.6:
            
            # 如果参数nans_convert_binary为True, 把存在的值替换成1，空值为0
            if nans_convert_binary == True:
                input_dataframe[column] = input_dataframe[column].notnull().astype('int')
            else:
                # 删除特征
                input_dataframe = drop_col(input_dataframe, column)
                remain_features.remove(column)
                continue;
                
        # 识别年份，特殊处理，转换为object类型，之后转换为枚举类
        input_dataframe = detect_year(input_dataframe, column)
            
        # 识别编号，特殊处理
        input_dataframe = detect_number(input_dataframe, column)
        
        # 如果存在空值需要对空值进行填补
        if input_dataframe[column].isnull().values.any():
            try:
                print(remain_features)
                if null_method != 'similarity':
                    input_dataframe,_ = deal_non(input_dataframe, column, null_method, similarity_features,remain_features)
                else:
                    input_dataframe = deal_non(input_dataframe, column, null_method, similarity_features,remain_features)

                unique = input_dataframe[column].unique()

                # 特征值唯一为无效特征
                if len(unique) == 1:
                    input_dataframe = drop_col(input_dataframe, column)
                    remain_features.remove(column)
                    continue; 

                # 删除异常值
                if str(input_dataframe[column].dtype) == 'int64' or str(input_dataframe[column].dtype) == 'float64':

                    # 若特征包含大量0，会导致25分位数和75分位数为0，需要特殊处理               
                    if zeros_ratio < 0.7:
                        # 删除异常值
                        if replace_outfiers_by_percent == False:
                            input_dataframe,_,_ = drop_outfiers(input_dataframe, column)
                            print(type(input_dataframe))
                        # 用极大极小值代替异常值
                        else:
                            input_dataframe[column],_,_ = replace_outfiers_by_percent(input_dataframe, column, percent)


            # 对于string类型的特征
            except TypeError:
                most_frequent = input_dataframe[column].mode()
                if len(most_frequent) > 0:
                    input_dataframe[column].fillna(input_dataframe[column].mode()[0], inplace=True)
                else:
                    input_dataframe[column].fillna(method='bfill', inplace=True)
        
        # 把所有string类型转换成枚举类型        
        try:
            input_dataframe[column].values.astype('float')

        except:
            # 对指定特征采用OneHot编码
            if one_hot_encoder == True:
                ret = list(set(input_dataframe.columns.tolist()).difference(set(no_one_hot_features)))
                if ret:
                    if column not in no_one_hot_features:
                        input_dataframe = one_hot_encoder_feature(input_dataframe, column)
                        
                    # 对未指定的特征使用LabelEncoder编码方式
                    else:
                        
                        input_dataframe[column] = label_encoder(input_dataframe, column)
                
                        
            # 使用LabelEncoder编码方式
            else:
                input_dataframe[column] = label_encoder(input_dataframe, column)
                
        # 是否归一化
        if normalization:
            input_dataframe[column],_,_ = normalize(input_dataframe, column, norm_max, norm_min)
    return input_dataframe


In [4]:
def autoclean_cv(input_dataframe, test_dataframe, normalization = False, drop_nans=False,
              nans_convert_binary = False, null_method = 'mode', 
              similarity_features = None, enum_continuous = False, 
              one_hot_encoder = False, no_one_hot_features = None, 
              norm_max = 1, norm_min = 0, 
              replace_outfiers_by_percent = False, percent = 95):
    """
    当输入数据集包含训练集和测试集时，需要对测试集采用和训练集同样的处理标准
    
    ----------
    input_dataframe: pandas.DataFrame
        输入的数据集
    test_dataframe: pandas.DataFrame
        输入的测试集
    normalization: bool
        对数据归一化 （默认: False）
    drop_nans: bool
        删除所有包含NaN的行 (默认: False)
    nans_convert_binary: bool
        把存在的值替换为1，不存在替换为0
    null_method: string
        空缺值处理方法 （默认: mode）
        可选 mode, average, median, similarity
    similarity_features: list
        以list中的特征作为相似度处理空缺值
    enum_continuous: bool
        把连续型变量转换为枚举类 （默认: True）
    one_hot_encoder: bool
        使用OneHotEncoder编码方式. (默认: False)
    no_one_hot_features: list
        指定不需要使用OneHotEncoder编码的特征 (默认：None)
    norm_min: int
        归一化后范围的下界 （默认：0）
    norm_max: int
        归一化后范围的上界 （默认：1）
    replace_outfiers_by_percent: bool
        用极大极小值代替指定头尾百分比的值 （默认：False）
    percent: float
        仅当replace_outfiers_by_percent为True时有效，前percent百分比
        和后（100-percent）百分比的值被认为异常值
    Returns
    ----------
    input_dataframe: pandas.DataFrame
        清理完成后的数据集
    """
    if set(input_dataframe.columns.values) != set(test_dataframe.columns.values):
        raise ValueError('训练集特征和测试集特征必须相同')
    
    # 剩下的特征名
    remain_features = input_dataframe.columns.tolist()
    test_remain_features = test_dataframe.columns.tolist()
        
    # 删除重复的特征
    input_dataframe = drop_dup_cols(input_dataframe)
    test_dataframe = drop_dup_cols(test_dataframe)
    
    # 删除包含NaN的行
    if drop_nans:
        input_dataframe = drop_nans(input_dataframe)
        test_dataframe = drop_nans(test_dataframe)
    
    # 对于每一个特征
    for column in input_dataframe.columns.values:
        print("当前特征", column)
        
        # 分别计算记录总条数，种类数，空值数，0的数量，唯一值比例，空值比例，值为0的比例
        count = input_dataframe[column].count()
        unique_sum = len(input_dataframe[column].unique())
        empty = input_dataframe[column].isnull().sum()
        zeros = (input_dataframe[column] == 0).sum()
        unique_ratio = unique_sum / count
        empty_ratio = empty / count
        zeros_ratio = zeros/count
        
        # 判断是否为唯一标示特征，即当特征的每一列都不同，特征无效
        if unique_ratio > 0.7:
            input_dataframe = drop_col(input_dataframe, column)
            remain_features.remove(column)
            # 若测试集中的此特征被删除，训练集也应当删除
            test_dataframe = drop_col(test_dataframe, column)
            test_remain_features.remove(column)
            continue;
            
        # 当特征每一列都相同，特征无效
        if unique_sum  == 1:
            input_dataframe = drop_col(input_dataframe, column)
            remain_features.remove(column)
            test_dataframe = drop_col(test_dataframe, column)
            test_remain_features.remove(column)
            continue;
        
        # 对于空值比例较高的特征（> 60%）
        if empty_ratio > 0.6:
            
            # 如果参数nans_convert_binary为True, 把存在的值替换成1，空值为0
            if nans_convert_binary == True:
                input_dataframe[column] = input_dataframe[column].notnull().astype('int')
                test_dataframe[column] = test_dataframe[column].notnull().astype('int')
            else:
                # 删除特征
                input_dataframe = drop_col(input_dataframe, column)
                remain_features.remove(column)
                test_dataframe = drop_col(test_dataframe, column)
                test_remain_features.remove(column)
                continue;
                
        # 识别年份，特殊处理，转换为object类型，之后转换为枚举类
        input_dataframe = detect_year(input_dataframe, column)
        test_dataframe = detect_year(test_dataframe, column)
            
        # 识别编号，特殊处理
        input_dataframe = detect_number(input_dataframe, column)
        test_dataframe = detect_number(test_dataframe, column)
                      
        # 空值填补
        try:
            input_dataframe,replace_value = deal_non(input_dataframe, column, null_method, similarity_features)
            # 测试集应该用训练集的方法填补
            # 当训练集拿同一个值填补时，测试集也应当用这个值
            if null_method in ['mean','mode','median']:
                test_dataframe[column].fillna(replace_value, inplace=True)
            # 当训练集用相似性填补时，测试集可以用自身数据集的相似度填补
            else:
                if column not in similarity_features and input_dataframe[column].dtype == 'int64' or input_dataframe[column].dtype == 'float64':
                    test_dataframe[column].fillna(data.groupby(similarity_features)[column].transform('mean'),inplace=True)
                else:
                    test_dataframe[column].fillna(input_dataframe[column].mode(), inplace=True)
            
            # 训练集和测试集中不同取值的个数
            unique = input_dataframe[column].unique().tolist()
            test_unique = test_dataframe[column].unique().tolist()
            
            # 特征值唯一为无效特征
            if len(unique) == 1:
                input_dataframe = drop_col(input_dataframe, column)
                remain_features.remove(column)
                test_dataframe = drop_col(test_dataframe, column)
                test_remain_features.remove(column)
                continue; 
                
            # 删除异常值
            if input_dataframe[column].dtype == 'int64' or input_dataframe[column].dtype == 'float64':
                
                # 若特征包含大量0，会导致25分位数和75分位数为0，需要特殊处理               
                if zeros_ratio < 0.7:
                    if replace_outfiers_by_percent == False:
                        drop_indices = []
                        input_dataframe, percent_25, percent_75 = drop_outfiers(input_dataframe, column)
                        # 测试集应当按照训练集的标准删除异常值
                        normal_range = (percent_75 - percent_25) * 3
                        outlier_indexs = test_dataframe[(test_dataframe[column] < percent_25 - normal_range) | (test_dataframe[column] > percent_75 + normal_range)].index
                        drop_indices.extend(list(outlier_indexs ))
                        test_dataframe = test_dataframe.drop(drop_indices)
                    else:
                        input_dataframe = replace_outfiers_by_percent(input_dataframe, column, percent)
                        # 测试集应当按照训练集的标准删除异常值
                        test_dataframe[column] = test_dataframe[column].map(lambda x: percent_max if x > percent_max else x)
                        test_dataframe[column] = test_dataframe[column].map(lambda x: percent_min if x < percent_min else x)
                        
                    
        
        # 对于string类型的特征
        except TypeError:
            # 当测试集中特征的出现了训练集中没有出现的取值时，应删掉
            unique = input_dataframe[column].unique().tolist()
            test_unique = test_dataframe[column].unique().tolist()
            if len(test_unique) > len(unique):
                test_dataframe = test_dataframe[test_dataframe[column].isin(unique)]
                print('delete successfully')
            
            # string类型的特征用出现最多的字符填补，测试集应该使用填补测试集的的字符填补
            most_frequent = input_dataframe[column].mode()
            if len(most_frequent) > 0:
                input_dataframe[column].fillna(input_dataframe[column].mode()[0], inplace=True)
                test_dataframe[column].fillna(input_dataframe[column].mode()[0], inplace=True)
            else:
                input_dataframe[column].fillna(input_dataframe[column].mode(), inplace=True)
                test_dataframe[column].fillna(input_dataframe[column].mode(), inplace=True)

        
        # 把所有string类型转换成枚举类型
        if str(input_dataframe[column].values.dtype) == 'object':
            # 对指定特征采用OneHot编码
            # 训练集和测试集应当使用相同的编码规则，否则当训练集的变量中多于测试集时，同一种变量的编码结果会不同
            if one_hot_encoder == True:
                ret = list(set(input_dataframe.columns.tolist()).difference(set(no_one_hot_features)))
                if ret:
                    if column not in no_one_hot_features:
                        input_dataframe = one_hot_encoder_feature(input_dataframe, column)
                        test_dataframe = one_hot_encoder_feature(test_dataframe, column)
                    # 对未指定的特征使用LabelEncoder编码方式
                    else:
                        le = LabelEncoder()
                        le.fit(input_dataframe[column].values.ravel())
                        keys = le.classes_
                        values = le.transform(le.classes_)
                        dictionary = dict(zip(keys, values))
                        input_dataframe[column] = input_dataframe[column].map(lambda x: dictionary[x])
                        test_dataframe[column] = test_dataframe[column].map(lambda x: dictionary[x])
                
                        
            # 使用LabelEncoder编码方式，训练集和测试集应当使用相同的编码规则
            else:
                le = LabelEncoder()
                le.fit(input_dataframe[column].values.ravel())
                keys = le.classes_
                values = le.transform(le.classes_)
                dictionary = dict(zip(keys, values))
                input_dataframe[column] = input_dataframe[column].map(lambda x: dictionary[x])
                test_dataframe[column] = test_dataframe[column].map(lambda x: dictionary[x])

                
        # 归一化
        if normalization:
            # 训练集归一化时储存min_x, max_x
            dict_norm = {}
            input_dataframe, min_x, max_x = normalize(input_dataframe, column, norm_min, norm_max)
            min_max_list = []
            min_max_list.append(min(input_dataframe[column]))
            min_max_list.append(max(input_dataframe[column]))
            dict_norm[column] = min_max_list
            # 对测试集采用相同的范围归一化
            range_norm = norm_max - norm_min
            range_x = dict_norm[column][1] - dict_norm[column][0]
            test_dataframe[column] = test_dataframe[column].map(lambda x: x/range_x * range_norm)

    return test_dataframe


### 使用Titanic数据集测试

In [5]:
input_dataframe = pd.read_csv("train.csv")
input_dataframe.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

对Sex特征使用OneHot编码

In [6]:
one_hot = ['Sex']

# 指定不适用OneHot编码的特征
no_one_hot_features = list(set(input_dataframe.columns.tolist()).difference(set(one_hot)))

cleaned_data = autoclean(input_dataframe,one_hot_encoder = True, no_one_hot_features = no_one_hot_features, 
                         null_method = 'mean')

当前特征 PassengerId
成功删除特征： PassengerId
当前特征 Survived
当前特征 Pclass
当前特征 Name
成功删除特征： Name
当前特征 Sex
当前特征 Age
['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
<class 'pandas.core.frame.DataFrame'>
当前特征 SibSp
当前特征 Parch
当前特征 Ticket
成功删除特征： Ticket
当前特征 Fare
当前特征 Cabin
成功删除特征： Cabin
当前特征 Embarked
['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']


In [7]:
# 自动清理完成的数据
cleaned_data.head()

female  male  Survived  Pclass   Age  SibSp  Parch     Fare  Embarked
0       0     1         0       3  22.0      1      0   7.2500       0.0
1       1     0         1       1  38.0      1      0  71.2833       1.0
2       1     0         1       3  26.0      0      0   7.9250       1.0
3       1     0         1       1  35.0      1      0  53.1000       1.0
4       0     1         0       3  35.0      0      0   8.0500       0.0

## 2. 使用卡方分箱对特征优化分箱，返回分箱信息

In [8]:
# 定义一个卡方分箱（可设置参数置信度水平与箱的个数）停止条件为大于置信水平且小于bin的数目
def ChiMerge(df, variable, flag, confidenceVal=3.841, bin=10, sample = None): 
    '''
    df:传入一个数据框仅包含一个需要卡方分箱的变量与正负样本标识（正样本为1，负样本为0）
    variable:需要卡方分箱的变量名称（字符串）
    flag：正负样本标识的名称（字符串）
    confidenceVal：置信度水平（默认是不进行抽样95%）
    bin：最多箱的数目
    sample: 为抽样的数目（默认是不进行抽样），因为如果观测值过多运行会较慢
    -----------------------
    return:
    result_data: dataframe
    包含变量名，分割间隔，每个间隔对应目标特征的数量
    
    '''
    # 进行是否抽样操作
    if sample != None:
        df = df.sample(n=sample)
    else:
        df   

    # 进行数据格式化录入
    
    # 统计需分箱变量每个值的数量
    total_num = df.groupby([variable])[flag].count()
    
    # 转换为dataframe形式
    total_num = pd.DataFrame({'total_num': total_num})
    
    # 统计需分箱变量每个值正样本数量
    positive_class = df.groupby([variable])[flag].sum()
    
    # 转换为dataframe形式
    positive_class = pd.DataFrame({'positive_class': positive_class})
    
    # 组合total_num与positive_class,DataFrame格式
    regroup = pd.merge(total_num, positive_class, left_index=True, right_index=True,
                       how='inner')
    
    # 增加索引列
    regroup.reset_index(inplace=True)
    
    # 增加列：分箱变量每个值负样本数量
    regroup['negative_class'] = regroup['total_num'] - regroup['positive_class'] 
    
    # 删除 'total_num' 列
    regroup = regroup.drop('total_num', axis=1)
    
    # 把数据框转化为numpy（提高运行效率）
    np_regroup = np.array(regroup)  

    # 处理连续没有正样本或负样本的区间，并进行区间的合并（以免卡方值计算报错）
    i = 0
    while (i <= np_regroup.shape[0] - 2):
        
        # 如果上下两行的正样本个数为0或负样本个数为0
        if ((np_regroup[i, 1] == 0 and np_regroup[i + 1, 1] == 0) or ( np_regroup[i, 2] == 0 and np_regroup[i + 1, 2] == 0)):
            np_regroup[i, 1] = np_regroup[i, 1] + np_regroup[i + 1, 1]  # 正样本
            np_regroup[i, 2] = np_regroup[i, 2] + np_regroup[i + 1, 2]  # 负样本
            np_regroup[i, 0] = np_regroup[i + 1, 0]
            
            # 删除第二行
            np_regroup = np.delete(np_regroup, i + 1, 0)
            i = i - 1
        i = i + 1
 
    # 对相邻两个区间进行卡方值计算
    chi_table = np.array([])  # 创建一个数组保存相邻两个区间的卡方值
    for i in np.arange(np_regroup.shape[0] - 1):
        chi = (np_regroup[i, 1] * np_regroup[i + 1, 2] - np_regroup[i, 2] * np_regroup[i + 1, 1]) ** 2 \
          * (np_regroup[i, 1] + np_regroup[i, 2] + np_regroup[i + 1, 1] + np_regroup[i + 1, 2]) / \
          ((np_regroup[i, 1] + np_regroup[i, 2]) * (np_regroup[i + 1, 1] + np_regroup[i + 1, 2]) * (
          np_regroup[i, 1] + np_regroup[i + 1, 1]) * (np_regroup[i, 2] + np_regroup[i + 1, 2]))
        chi_table = np.append(chi_table, chi)

    # 把卡方值最小的两个区间进行合并（卡方分箱核心）
    while (1):     
        # 结束循环条件
        if (len(chi_table) <= (bin - 1) and min(chi_table) >= confidenceVal):
            break
            
        # 找出卡方值最小的位置索引
        chi_min_index = np.argwhere(chi_table == min(chi_table))[0]  
        np_regroup[chi_min_index, 1] = np_regroup[chi_min_index, 1] + np_regroup[chi_min_index + 1, 1]
        np_regroup[chi_min_index, 2] = np_regroup[chi_min_index, 2] + np_regroup[chi_min_index + 1, 2]
        np_regroup[chi_min_index, 0] = np_regroup[chi_min_index + 1, 0]
        np_regroup = np.delete(np_regroup, chi_min_index + 1, 0)

        if (chi_min_index == np_regroup.shape[0] - 1):  # 最小值试最后两个区间的时候
            # 计算合并后当前区间与前一个区间的卡方值并替换
            chi_table[chi_min_index - 1] = (np_regroup[chi_min_index - 1, 1] * np_regroup[chi_min_index, 2] - np_regroup[chi_min_index - 1, 2] * np_regroup[chi_min_index, 1]) ** 2 \
                                           * (np_regroup[chi_min_index - 1, 1] + np_regroup[chi_min_index - 1, 2] + np_regroup[chi_min_index, 1] + np_regroup[chi_min_index, 2]) / \
                                       ((np_regroup[chi_min_index - 1, 1] + np_regroup[chi_min_index - 1, 2]) * (np_regroup[chi_min_index, 1] + np_regroup[chi_min_index, 2]) * (np_regroup[chi_min_index - 1, 1] + np_regroup[chi_min_index, 1]) * (np_regroup[chi_min_index - 1, 2] + np_regroup[chi_min_index, 2]))
            # 删除替换前的卡方值
            chi_table = np.delete(chi_table, chi_min_index, axis=0)

        else:
            # 计算合并后当前区间与前一个区间的卡方值并替换
            chi_table[chi_min_index - 1] = (np_regroup[chi_min_index - 1, 1] * np_regroup[chi_min_index, 2] - np_regroup[chi_min_index - 1, 2] * np_regroup[chi_min_index, 1]) ** 2 \
                                       * (np_regroup[chi_min_index - 1, 1] + np_regroup[chi_min_index - 1, 2] + np_regroup[chi_min_index, 1] + np_regroup[chi_min_index, 2]) / \
                                       ((np_regroup[chi_min_index - 1, 1] + np_regroup[chi_min_index - 1, 2]) * (np_regroup[chi_min_index, 1] + np_regroup[chi_min_index, 2]) * (np_regroup[chi_min_index - 1, 1] + np_regroup[chi_min_index, 1]) * (np_regroup[chi_min_index - 1, 2] + np_regroup[chi_min_index, 2]))
            # 计算合并后当前区间与后一个区间的卡方值并替换
            chi_table[chi_min_index] = (np_regroup[chi_min_index, 1] * np_regroup[chi_min_index + 1, 2] - np_regroup[chi_min_index, 2] * np_regroup[chi_min_index + 1, 1]) ** 2 \
                                       * (np_regroup[chi_min_index, 1] + np_regroup[chi_min_index, 2] + np_regroup[chi_min_index + 1, 1] + np_regroup[chi_min_index + 1, 2]) / \
                                   ((np_regroup[chi_min_index, 1] + np_regroup[chi_min_index, 2]) * (np_regroup[chi_min_index + 1, 1] + np_regroup[chi_min_index + 1, 2]) * (np_regroup[chi_min_index, 1] + np_regroup[chi_min_index + 1, 1]) * (np_regroup[chi_min_index, 2] + np_regroup[chi_min_index + 1, 2]))
            # 删除替换前的卡方值
            chi_table = np.delete(chi_table, chi_min_index + 1, axis=0)

    # 把结果保存成一个数据框
    result_data = pd.DataFrame()  # 创建一个保存结果的数据框
    result_data['variable'] = [variable] * np_regroup.shape[0]  # 结果表第一列：变量名
    list_temp = []
    for i in np.arange(np_regroup.shape[0]):
        if i == 0:
            x = '0' + ',' + str(np_regroup[i, 0])
        elif i == np_regroup.shape[0] - 1:
            x = str(np_regroup[i - 1, 0])
        else:
            x = str(np_regroup[i - 1, 0]) + ',' + str(np_regroup[i, 0])
        list_temp.append(x)
    result_data['interval'] = list_temp  # 结果表第二列：区间
    result_data['flag_0'] = np_regroup[:, 2]  # 结果表第三列：负样本数目
    result_data['flag_1'] = np_regroup[:, 1]  # 结果表第四列：正样本数目

    return result_data

## 3. 根据分箱信息对指定变量分箱操作

In [9]:
def chi_square_bin(cleaned_data, bin_features, flag):
    '''
    cleaned_data: dataframe
    已经自动清理完成的数据
    bin_features: list
    需要分箱操作的特征名
    flag: string
    目标特征
    '''
    
    # 对每一个特征进行分箱操作
    for feature in bin_features:
        
        # 返回分割信息
        result= ChiMerge(cleaned_data,feature, flag, confidenceVal=3.841, bin=10)
        
        # 得到分割间隔
        res = result['interval'].tolist()
        eles = []
        for ele in res:
            tmp = ele.split(",")
            for e in tmp:
                eles.append(float(e))
        seen = set()
        bins = tuple(x for x in eles if not (x in seen or seen.add(x)))
        
        # 根据分割间隔把特征转换成枚举类
        categories = pd.cut(input_dataframe[feature],bins)
        cleaned_data[feature] = categories.cat.codes 
    return cleaned_data

In [10]:
bin_features = ['Age','Fare']
flag = 'Survived'
cleaned_data = chi_square_bin(cleaned_data, bin_features, flag)
cleaned_data.head()

female  male  Survived  Pclass  Age  SibSp  Parch  Fare  Embarked
0       0     1         0       3    3      1      0     0       0.0
1       1     0         1       1    6      1      0     8       1.0
2       1     0         1       3    3      0      0     3       1.0
3       1     0         1       1    6      1      0     7       1.0
4       0     1         0       3    6      0      0     3       0.0

## 4. 根据IV值筛选出重要的特征

In [11]:
class WOE:
    '''
    根据IV值筛选出重要性大的特征
    输入：自动清理完成的数据集
    输出：每个特征的IV值
    '''
    def __init__(self):
        self._WOE_MIN = -20
        self._WOE_MAX = 20

    # 计算输入数据中每个变量的WOE和IV
    def woe(self, X, y, event=1):
        '''
        X: 2-D numpy array
        包含已经离散化的特征
        y: 1-D numpy array
        目标特征
        :event: 目标特征的其中一个值
        Return: 
        -----------
        res_iv: np.array
            每个特征的IV值
        res_woe: np.array
            每个特征每个类的WOE值
            
        '''
        # 检查y是否为binary变量
        self.check_target_binary(y)
        
        # 离散化连续型变量 (自动清理阶段应对连续性变量使用优化分箱离散)
        X1 = self.feature_discretion(X)
        
        # 分别储存WOE, IV值
        res_woe = []
        res_iv = []
        
        # 对于每一条记录储存WOE, IV值
        for i in range(0, X1.shape[-1]):
            x = X1[:, i]
            woe_dict, iv1 = self.woe_single_x(x, y, event)
            res_woe.append(woe_dict)
            res_iv.append(iv1)
        return np.array(res_woe), np.array(res_iv)

    # 对于每一个特征计算WOE和IV值
    def woe_single_x(self, x, y, event=1):

        # 检查目标特征是否为binary形式
        self.check_target_binary(y)
        
        # 得到目标变量为1和0的数量
        event_total, non_event_total = self.count_binary(y, event=event)
        
        # 特征取不同值个数
        x_labels = np.unique(x)
        woe_dict = {}
        iv = 0
        for x1 in x_labels:
            y1 = y[np.where(x == x1)[0]]
            # 在该取值时目标变量为1和0的数量和比例
            event_count, non_event_count = self.count_binary(y1, event=event)
            rate_event = 1.0 * event_count / event_total
            rate_non_event = 1.0 * non_event_count / non_event_total
            if rate_event == 0:
                woe1 = self._WOE_MIN
            elif rate_non_event == 0:
                woe1 = self._WOE_MAX
            else:
                woe1 = math.log(rate_event / rate_non_event)
            woe_dict[x1] = woe1
            # 根据WOE值计算IV值
            iv += (rate_event - rate_non_event) * woe1
        return woe_dict, iv


    def count_binary(self, a, event=1):
        '''
        计算目标特征分别为0或1的个数
        '''
        event_count = (a == event).sum()
        non_event_count = a.shape[-1] - event_count
        return event_count, non_event_count

    def check_target_binary(self, y):
        '''
        检查目标特征是否为binary形式，否则抛出异常
        -----------
        y: 1-D numpy array
        '''
        y_type = type_of_target(y)
        if y_type not in ['binary']:
            raise ValueError('目标特征必须为binary类型')

    def feature_discretion(self, X):
        '''
        离散化连续性变量，保持其他类型变量不变
        :param X : numpy array
        :return: the numpy array 
            所有连续性变量已离散化
        '''
        temp = []
        for i in range(0, X.shape[-1]):
            x = X[:, i]
            x_type = type_of_target(x)
            if x_type == 'continuous':
                x1 = self.discrete(x)
                temp.append(x1)
            else:
                temp.append(x)
        return np.array(temp).T

    def discrete(self, x):
        '''
        离散化特征，分成等间距的5个区间
        :param x: 1-D numpy array
        :return: discreted 1-D numpy array
        '''
        res = np.array([0] * x.shape[-1], dtype=int)
        for i in range(5):
            point1 = stats.scoreatpercentile(x, i * 20)
            point2 = stats.scoreatpercentile(x, (i + 1) * 20)
            x1 = x[np.where((x >= point1) & (x <= point2))]
            mask = np.in1d(x, x1)
            res[mask] = (i + 1)
        return res

    @property
    def WOE_MIN(self):
        return self._WOE_MIN
    @WOE_MIN.setter
    def WOE_MIN(self, woe_min):
        self._WOE_MIN = woe_min
    @property
    def WOE_MAX(self):
        return self._WOE_MAX
    @WOE_MAX.setter
    def WOE_MAX(self, woe_max):
        self._WOE_MAX = woe_max

## 5. 根据IV值保留重要的特征

In [12]:
def keep_good_features(cleaned_data,target_feature = cleaned_data.columns.values.tolist()[-1],importance = 0.15):
    '''
    根据IV值保留重要的特征
    输入：自动清理完成的数据集
    输出：删除不重要特征的数据集
    '''
    
    target = cleaned_data[target_feature].values
    
    # 构建WOE实例
    test = WOE()
    
    target_data = cleaned_data[target_feature].values
    features_data = cleaned_data.drop([target_feature],axis = 1).values
    res_woe, res_iv = test.woe(features_data, target_data, event=1)

    iv_list = res_iv.tolist()

    features = cleaned_data.columns.values.tolist()


    fea_iv_dict = dict(zip(features,iv_list))

    remain_fea_iv_dict = dict((key, value) for key, value in fea_iv_dict.items() if value > importance)

    remainds = []
    for key, value in remain_fea_iv_dict.items():
        remainds.append(key) 

    for feature in cleaned_data.columns.values.tolist():
        if not feature in remainds:
            remaind_data = cleaned_data.drop([feature],axis = 1)
            
    return remaind_data

In [13]:
remaind_data = keep_good_features(cleaned_data,target_feature = 'Survived')

In [14]:
remaind_data.head()

female  male  Survived  Pclass  Age  SibSp  Parch  Fare
0       0     1         0       3    3      1      0     0
1       1     0         1       1    6      1      0     8
2       1     0         1       3    3      0      0     3
3       1     0         1       1    6      1      0     7
4       0     1         0       3    6      0      0     3

## 6. 对自动清理的数据集使用LR模型预测

In [15]:
y = cleaned_data['Survived']
X = cleaned_data.drop(['Survived'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
classifier = LogisticRegression() 
classifier.fit(X_train, y_train) 
prediction = classifier.predict(X_test)
accuracy = accuracy_score(y_test,prediction)
print(accuracy)

0.8163265306122449


## 7. 自动调参 （暂时未装autosklearn）

In [ ]:
import autosklearn.classification
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics

In [ ]:
automl = autosklearn.classification.AutoSklearnClassifier()
automl.fit(X_train, y_train)
y_hat = automl.predict(X_test)

# 得到调参完成后的准确率
print("Accuracy score", sklearn.metrics.accuracy_score(y_test, y_hat))

In [ ]:
# ensemble model 中每个模型的权重
automl.get_models_with_weights()

In [ ]:
# ensemble model 中每个模型的参数选择
automl.show_models()